In [1]:
import tensorflow as tf

import energy_py
from energy_py.agents import DQN, tfValueFunction
from energy_py.envs import BatteryEnv

Using TensorFlow backend.


In [2]:
EPISODE_LENGTH = 48
BRAIN_PATH = '/'
BATCH_SIZE=64
EPISODES = 10
DISCOUNT = 0.9
total_steps = EPISODES * EPISODE_LENGTH

%load_ext line_profiler

In [3]:
env = BatteryEnv('/home/adam/git/energy_py/energy_py/experiments/battery/', episode_length=48)

In [4]:
agent = DQN(env, 
            DISCOUNT, 
            brain_path='/',
            Q=tfValueFunction,
            batch_size=BATCH_SIZE,
            total_steps=total_steps)

In [5]:
def single_ep():
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        #  initialize before starting episode
        done, step, total_step = False, 0, 0
        episode = 1
        observation = env.reset(episode)

        #  while loop runs through a single episode
        while done is False:
            #  select an action
            action = agent.act(sess=sess, obs=observation)
            #  take one step through the environment
            next_observation, reward, done, info = env.step(action)
            #  store the experience
            agent.memory.add_experience(observation, action, reward,
                                        next_observation, done, step, episode)
            step += 1
            total_step += 1
            observation = next_observation

            #  with DQN we can learn within episode 
            #  get a batch to learn from
            batch = agent.memory.get_random_batch(BATCH_SIZE)

            train_info = agent.learn(sess=sess, batch=batch)

            if total_step % agent.update_target_net == 0:
                agent.update_target_network(sess)
    return env, agent

In [6]:
single_ep()

inputs shape (1, 6)
Q preds for next_obs shape (1, 100)
max q values has shape (1,)
[False]
target shape after bellman equn (1,)
target shape processing (1, 1)


ValueError: [0.06387118590866603, 0.32575804974146894] is not in list